In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("dataproc_spark") \
    .getOrCreate()

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("iamsouravbanerjee/world-population-dataset")
df = SparkSession.read.csv(path, header=True, inferSchema=True)
df.head()
print("First 5 records:", df.head(5))

AttributeError: 'property' object has no attribute 'csv'

In [ ]:
!pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "/udayka/world_population.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "iamsouravbanerjee/world-population-dataset",
  file_path,
)

print("First 5 records:", df.head())

<ipython-input-5-44272ba58397>:8: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


KaggleApiHTTPError: 404 Client Error.

Resource not found at URL: https://www.kaggle.com/datasets/iamsouravbanerjee/world-population-dataset/versions/4
The server reported the following issues: Dataset not found
Please make sure you specified the correct resource identifiers.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
csv_file_url = "https://raw.githubusercontent.com/databricks/LearningSparkV2/master/chapter3/data/retail-data/all/online-retail-dataset.csv"
df = spark.read.csv(csv_file_url, header=True, inferSchema=True)
print("Original DataFrame:")
df.show(5)
print("Schema:")
df.printSchema()
print(f"Number of records: {df.count()}")

# 1. Select specific columns
selected_df = df.select("InvoiceNo", "StockCode", "Description")
print("\nSelected columns:")
selected_df.show(5)


# 2. Filter rows based on condition
filtered_df = df.filter(col("Quantity") > 10)
print("\nFiltered rows (Quantity > 10):")
filtered_df.show(5)

# 3. Distinct values in a column
distinct_stockcodes = df.select("StockCode").distinct()
print("\nDistinct StockCodes:")
distinct_stockcodes.show(5)

# 4. Group By and aggregation
grouped_df = df.groupBy("StockCode").agg(count("*").alias("Count"))
print("\nGrouped by StockCode and aggregated:")
grouped_df.show(5)

# 5. Order by
ordered_df = df.orderBy(desc("Quantity"))
print("\nOrdered by Quantity in descending order:")
ordered_df.show(5)

# 6. Add new Column
df_new_col = df.withColumn('New_col',lit(5))
print("\nAdded new column :")
df_new_col.show(5)

# 7. Rename columns
df_renamed = df.withColumnRenamed("InvoiceNo","Invoice_Number")
print("\nRenamed InvoiceNo to Invoice_Number:")
df_renamed.show(5)

# 8. Drop columns
df_dropped = df.drop("InvoiceNo","StockCode")
print("\nDroped columns InvoiceNo and StockCode :")
df_dropped.show(5)

# Stop Spark Session
# spark.stop()


UnsupportedOperationException: None